# Row Echelon Form Generator

In this project we want to enter a matrix $A \in \mathrm{Mat}( n \times m; \mathbb{Q} )$ of any $n,m \in \mathbb{N}$, calculate which row operations must be executed to reach the row echelon form $A'$ and then output a matrix $E$, so that $A' = E * A$, as well as the different row operations.

### The MATH behind it

Every row operation can be viewed as a left multiplication with an elementary matrix. There are the following three row operations with their corresponding matrix:

First let's define a matrix $E^{(i,j)} \in \mathrm{Mat}( n \times n; \mathbb{Q} )$, where $l,k \in \{1, ..., n\}$:
$$
\begin{equation*}
    E^{(i,j)} := (e^{(i,j)}_{p,q})_{\substack{1 \leq p \leq n \\ 1 \leq q \leq n}} \text{ where }
    e^{(i,j)}_{p,q} := 
    \begin{cases}
        1 \text{, if } p=k \text{ and } q=l\\
        0 \text{, else }\\
    \end{cases}
\end{equation*}
$$

Then we can now define our elementary matrices $\left(i \neq j \in \{1, ..., n\}, \lambda \in \mathbb{Q} - \{0\}\right)$.\
The First one is **S**wapping two rows $R_i \leftrightarrow R_j$:
$$
\begin{equation*}
    S^{(i,j)} := 1_n - E^{(i,i)}- E^{(j,j)} + E^{(i,j)} + E^{(j,i)}
\end{equation*}
$$

The Second one is **M**ultiplying a row by a nonzero number $\lambda R_i \rightarrow R_i$:
$$
\begin{equation*}
    M^{(i)}_{\lambda} := 1_n + (\lambda - 1) E^{(i,i)}
\end{equation*}
$$

The Third one is **A**dding a multiple of one row to another row $\lambda R_i + R_j \rightarrow R_j$:
$$
\begin{equation*}
    A^{(i,j)}_{\lambda} := 1_n + \lambda E^{(i,j)}
\end{equation*}
$$

Let now $F := \begin{cases} S^{(i,j)} \\  M^{(i)}_{\lambda} \\ A^{(i,j)}_{\lambda} \\ \end{cases} $\
Because we just have to apply the three row operations to a matrix to receive the row echelon form, we can conclude the following, where  $a$ is the total number of applied row operations and $F_b, b \in \{1,...,a\}$ is the corresponding elementary matrix.
$$
\begin{equation*}
    A' = F_a * (F_{a-1} * (... * (F_1 * A)...))
\end{equation*}
$$

Due to the ~~commutativity~~ associativity of matrix multiplication we can now conclude:
$$
\begin{align*}
    A' &= F_a * F_{a-1} * ... * F_1 * A \\
       &= (F_a * F_{a-1} * ... * F_1) * A \\
       &= E * A
\end{align*}
$$
Of course $E := (F_a * F_{a-1} * ... * F_1)$. So now we found out how to calculate our matrix $E$.

### Initiate matrix

You have to input the matrix $A$ manually. Choose any dimensions, but values are in $\mathbb{Q}$.

In [1]:
from IPython.display import Latex as tex

# input matrix A manually
A = matrix(QQ, [[0, 2, 3], [4, 5, 6], [7, 8, 9]])
#A = matrix(QQ, [[4, 6, 1, 9, 4, 2, 7], [3, 8, 2, 5, 1, 0, 6], [7, 8, 9, 0, 3, 4, 2], [9, 0, 6, 3, 4, 7, 5]])
# calculate the number of rows
n = A.nrows()

# print the matrix and save it as M to operate on
show(A)
M = A

# create the matrix E and set it to an nxn identity matrix 
E = identity_matrix(n)

[0 2 3]
[4 5 6]
[7 8 9]

### Test if a matrix is in row-echelon form (function)

As a little subproject we also defined a function which calculates whether a given matrix is in row echelon form. Our solution to that is quite neat.

In [2]:
# function that tests whether a matrix B is in row echelon form 

def is_in_row_ech_form(B):
    # sage gives you the columns of the pivots, we add an imaginary extra pivot, so we can always calculate the difference to the next pivot, or the end of the matrix
    pivots = list(B.pivots())
    pivots.append(B.ncols())

    # iterate over the pivot columns
    for i in range(0, len(pivots)-1):
        # generate a zero matrix with k columns and l rows, where k is the difference between the currently observed pivot and the next pivot, and l is the number of rows of B minus the row after the currently observed pivot (i+1)
        Zero = zero_matrix(B.nrows() - i - 1, pivots[i+1] - pivots[i])

        # define a submatrix that reaches from under the current pivot to the next one
        matrix_under_pivot = B[i+1:,pivots[i]:pivots[i+1]]
        
        # test if this submatrix is a zero matrix
        if(Zero != matrix_under_pivot):
            return(False)
    return(True)

In [3]:
# print wheter the matrix A is in row echelon form or not
show(is_in_row_ech_form(A))

False

### Calculate the row echelon form

As described in the MATH part, we can now just calculate $E$ by finding out which row operation should be the next. We then left multiply the elementary matrix with our matrix $A$ in order to know which operations have been done already.

In [4]:
# initiate the list of pivots and the array for the row operations
pivots = list(M.pivots())
operations = []

In [5]:
for i in range(0, len(pivots)):
    pivots[i]
    # first we check if number where the pivot should be equals zero. if so we swap it with the next row
    if(M[i , pivots[i]] == 0):
        for j in range(i+1, n):
            if(M[j , pivots[i]] != 0):
                F = elementary_matrix(n, row1=i, row2=j )
                M = F * M
                E = F * E
                ##operations.append("swap row " + str(i) + " with row " + str(i))
                operations.append("$R_{" + str(i+1) + "} \\leftrightarrow R_{" + str(j+1) + "}$")
                break
    
    # if the pivot element does not equal 1 we multiply the row by the inverse to make following operations easier
    if(M[i , pivots[i]] != 1):
        denomenator = M[i , pivots[i]]
        F = elementary_matrix(n, row1=i, scale=(1 / denomenator) )
        M = F * M
        E = F * E
        #operations.append("multiply row1 with inverse of pivot")
        operations.append("$" + latex(1 / denomenator) + " R_{" + str(i+1) + "} \\rightarrow R_{" + str(i+1) + "}$")
    
    # at last we make everything under the pivot zero
    for j in range(i+1, n): #Null-mach-Schleife
        factor = -M[j , pivots[i]]
        F = elementary_matrix(n, row1=j, row2=i, scale=factor)
        M = F * M
        E = F * E
        #operations.append("add row1 times factor to row2")
        operations.append("$" + latex(factor) + " R_{" + str(i+1) + "} \\rightarrow R_{" + str(j+1) + "}$")

### Output the results

In [6]:
show("Matrix A', the row echelon form of A:")
show(M)

show("Matrix E, with A' = E * A:")
show(E)

show("Test, that E * A = A' is true")
show(E*A==M)

"Matrix A', the row echelon form of A:"

[  1 5/4 3/2]
[  0   1 3/2]
[  0   0   1]

"Matrix E, with A' = E * A:"

[   0  1/4    0]
[ 1/2    0    0]
[  -1 14/3 -8/3]

"Test, that E * A = A' is true"

True

In [7]:
# show the operations that have been used
op_total = ""

for op in operations:
    op_total += (op + "\n\n")

tex(op_total)

<IPython.core.display.Latex object>